# TP2 - EA3
Diseñar, simular e implemetar un amplificador de microondas de baja señal para máxima transferencia de energía (adaptación de impedancias de entrada y salida)
Los requisitos son:
- $Z_{in}$ = 50 $\Omega$
- $Z_L$ = 50 $\Omega$
- FR4 $\rightarrow \epsilon_r = 4.5$
- El transistor elegido es BFP 450
 A continuacion se cargan en un arreglo los parámetros S en base a la tensión y corriente de polarización seleccionadas para el transistor

### Se procede a cargar los datos del archivo s2p para poder trabajar con ellos

In [30]:
import os
import numpy as np
import skrf as skrf

#Calcula los parámetros S en una frecuencia dada para cada archivo .s2p en las carpetas especificadas.
def calcular_parametros_s(frecuencia, carpetas):
    s_parameters = {}
    
    # Convertir frecuencia de GHz a Hz
    frecuencia *= 1e9                                  
    
    for carpeta in carpetas:
        archivos_s2p = [f for f in os.listdir(carpeta) if f.endswith('.s2p')]
        for archivo_s2p in archivos_s2p:
            ruta_archivo = os.path.join(carpeta, archivo_s2p)
            red = skrf.network.Network(ruta_archivo)
            
            # Encontrar la frecuencia más cercana en el archivo .s2p
            indice_frecuencia = np.argmin(np.abs(red.f - frecuencia))
                        
            # Acceder a los parámetros S en la frecuencia encontrada
            if archivo_s2p not in s_parameters:
                s_parameters[archivo_s2p] = red.s[indice_frecuencia]
    
    return s_parameters

### Se calcula el determinante para cada conjunto de parametros S

In [31]:
def calcular_delta(parametros_s):
    deltas = {}
    for archivo, parametros in parametros_s.items():
        delta = (parametros[0, 0] * parametros[1, 1]) - (parametros[0, 1] * parametros[1, 0])
        deltas[archivo] = delta

    return deltas

### Se calcula el parametro k de cada conjunto y se filtran los k > 1

In [32]:
def calcular_k(s_parametros, deltas):
    ks = {}
    polarizacion_k_bueno = {}
    for archivo, parametros in s_parametros.items():
        S11 = parametros[0, 0]
        S12 = parametros[0, 1]
        S21 = parametros[1, 0]
        S22 = parametros[1, 1]
        abs_delta = abs(deltas[archivo])
        
        # Calcular el parámetro k
        numerador = (1 - abs(S11)**2 - abs(S22)**2 + abs_delta**2)
        denominador = 2 * abs(S12 * S21)
        k = numerador/denominador
        
        ks[archivo] = k
        if k > 1:
            # Guardar la polarización si k es mayor que 1
            polarizacion_k_bueno[archivo] = {'k': k}
    
    return ks, polarizacion_k_bueno


### Se calculan los coeficientes B1, B2, C1 y C2 y las impedancias Zin y Zout

In [33]:
def calcular_impedancias(Zo, s_parameters, polarizaciones_buenas_k, ic_filter, frecuencia):
    for nombre_archivo, polarizacion in polarizaciones_buenas_k.items():
        # Obtener el valor de IC del nombre del archivo
        ic  = float(nombre_archivo.split('_IC_')[1].split('mA')[0])
        vce = float(nombre_archivo.split('_VCE_')[1].split('V')[0])
        impedancias = []

        # Filtro de polarizaciones
        if ic < ic_filter:
            # Obtener los parámetros S de la polarización actual
            parametros_s = s_parameters[nombre_archivo]

            # Asignar los valores de los parámetros S a las variables correspondientes
            S11 = parametros_s[0, 0]
            S12 = parametros_s[0, 1]
            S21 = parametros_s[1, 0]
            S22 = parametros_s[1, 1]
            det = S11 * S22 - S12 * S21                 # Determinate

            # Cálculo de B1, B2, C1, C2
            B1 = 1 + abs(S11)**2 - abs(S22)**2 - abs(det)**2
            B2 = 1 + abs(S22)**2 - abs(S11)**2 - abs(det)**2
            C1 = S11 - det * np.conj(S22)
            C2 = S22 - det * np.conj(S11)
            
            # Cálculo de gamma in
            gamma_in = (B1 - np.sqrt((B1**2) - (4 * (abs(C1)**2)))) / (2 * abs(C1))
            gamma_in_rect = gamma_in * np.cos(np.angle(C1)) + gamma_in * np.sin(np.angle(C1)) * 1j

            # Cálculo de gamma out
            gamma_out = (B2 - np.sqrt((B2**2) - (4 * (abs(C2)**2)))) / (2 * abs(C2))
            gamma_out_rect = gamma_out * np.cos(np.angle(C2)) + gamma_out * np.sin(np.angle(C2)) * 1j

            # Cálculo impedancia de entrada
            Z_in = Zo * ((1 + gamma_in_rect) / (1 - gamma_in_rect))
            condition_in = "L" if Z_in.imag >= 0 else "C"

            # Cálculo impedancia de salida
            Z_out = Zo * ((1 + gamma_out_rect) / (1 - gamma_out_rect))            
            condition_out = "L" if Z_out.imag >= 0 else "C"

               
            # Imprimir los resultados para cada configuración
            print(f"\033[1m\033[31mPolarización: Vce = {vce} [V] - Ic = {ic} [mA]\033[0m")
            print(f"S11: {S11:.4f}, S12: {S12:.4f}, S21: {S21:.4f}, S22: {S22:.4f}")
            print(f"B1 =", "{:.4f}".format(abs(B1)), "{:.4f}".format(np.degrees(np.angle(B1))), "°")
            print(f"B2 =", "{:.4f}".format(abs(B2)), "{:.4f}".format(np.degrees(np.angle(B2))), "°")
            print(f"C1 =", "{:.4f}".format(abs(C1)), "{:.4f}".format(np.degrees(np.angle(C1))), "°")
            print(f"C2 =", "{:.4f}".format(abs(C2)), "{:.4f}".format(np.degrees(np.angle(C2))), "°")
            print("Γᵢₙ      = ","{:.4f}".format(gamma_in), "{:.4f}".format(np.degrees(np.angle(C1))),"°")
            print("Γᵢₙ_rect = ","{:.4f}".format(gamma_in_rect))
            print("Γₒᵤ      = ","{:.4f}".format(gamma_out), "{:.4f}".format(np.degrees(np.angle(C2))),"°")
            print("Γₒᵤ_rect = ","{:.4f}".format(gamma_out_rect))
            print("Z_in     = ","{:.4f}".format(Z_in) , " - ", condition_in)
            print("Z_out    = ","{:.4f}".format(Z_out), " - ", condition_out)  
            print()

            impedancias.append((Z_in, Z_out))
    
    return impedancias



### Diseño de microtiras

In [34]:
#A_in = 

### Ejemplo de uso

In [42]:
frecuencia = 1.6                # GHz
Zo         = 50                 # Impedancia  
carpetas   = ["Datasheets/BFP640"]    # Rutas a las carpetas que contienen los archivos .s2p
ic_filter  = 50               # Corriente para filtrar polarizaciones

s_parameters = calcular_parametros_s(frecuencia, carpetas)
deltas       = calcular_delta(s_parameters)
ks, polarizacion_k_bueno = calcular_k(s_parameters, deltas)
impedancias  = calcular_impedancias(Zo, s_parameters, polarizacion_k_bueno, ic_filter, frecuencia)

Polarización: Vce = 1.5 [V] - Ic = 37.0 [mA]
S11: -0.3731+0.0117j, S12: 0.0225+0.0361j, S21: 1.9106+11.5409j, S22: 0.1200-0.1726j
B1 = 0.9170 0.0000 °
B2 = 0.7266 0.0000 °
C1 = 0.4583 -178.2623 °
C2 = 0.3631 -47.2490 °
Γᵢₙ      =  0.9716 -178.2623 °
Γᵢₙ_rect =  -0.9711-0.0295j
Γₒᵤ      =  0.9643 -47.2490 °
Γₒᵤ_rect =  0.6545-0.7081j
Z_in     =  0.7211-0.7581j  -  C
Z_out    =  5.6564-114.0767j  -  C

Polarización: Vce = 1.0 [V] - Ic = 45.0 [mA]
S11: -0.7102+0.2349j, S12: 0.0292+0.0392j, S21: 0.7264+2.6554j, S22: -0.0146+0.0117j
B1 = 1.5371 0.0000 °
B2 = 0.4188 0.0000 °
C1 = 0.7452 161.6835 °
C2 = 0.0925 -33.2641 °
Γᵢₙ      =  0.7792 161.6835 °
Γᵢₙ_rect =  -0.7397+0.2449j
Γₒᵤ      =  0.2328 -33.2641 °
Γₒᵤ_rect =  0.1947-0.1277j
Z_in     =  6.3635+7.9336j  -  L
Z_out    =  71.1291-19.2095j  -  C

Polarización: Vce = 1.0 [V] - Ic = 48.0 [mA]
S11: -0.7208+0.2609j, S12: 0.0290+0.0399j, S21: 0.7022+2.3556j, S22: -0.0285+0.0195j
B1 = 1.5647 0.0000 °
B2 = 0.3917 0.0000 °
C1 = 0.7615 160.0907 °